In [1]:
# -*- coding: utf-8 -*-

__author__ = 'Oswaldo Ludwig'
__version__ = '1.01'

import numpy as np
np.random.seed(42)  # for reproducibility
import pandas as pd
import os
import csv
import nltk
import itertools
import operator
import pickle
import numpy as np    
from keras.preprocessing import sequence
from scipy import sparse, io
from numpy.random import permutation
import re

from sklearn import preprocessing

from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Bidirectional, Dropout, merge
from keras.layers import Concatenate
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


import keras.backend as K
# import theano.tensor as T
import sys
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

df_hansard  = pd.read_csv('../../minerva/data/hansard_full.csv')

df_group = df_hansard.groupby('subjectOfBusinessId')

# for i in range(20):
#     print(i, df_hansard.iloc[i]['content'])
#     print('-' * 50)

q_a = []
for i, index in df_group.groups.items():
#     print(i, index)
    # don't bother with odd pairs
    if (len(index) % 2 != 0): 
        continue
        
#     print(list(index))
    t = df_hansard.iloc[list(index)]['content'].values
#     print(t)
#     print(list(zip(t[::2], t[1::2])))
    q_a.append(list(zip(t[::2], t[1::2])))

q_a = [item for sublist in q_a for item in sublist]

q_a = q_a[:2000]

print('number of q & a', len(q_a))
    
df_q_a = pd.DataFrame(q_a)
df_q_a.columns = ['Q', 'A']
df_q_a.to_csv('data/q_a.csv')
df_q_a.tail()

number of q & a 2000


,Q,A
1995,"Mr. Speaker, I can imagine the conversation th...","Mr. Speaker, we are the ones who put in place ..."
1996,"Mr. Speaker, Ms. Meilleur demonstrated that sh...","Mr. Speaker, after 10 years of petty politics ..."
1997,"Mr. Speaker, in Quebec, the law is clear. A ba...","Mr. Speaker, we want to ensure that Canadian c..."
1998,"Mr. Speaker, I will now read a motion that the...","Mr. Speaker, we want to be very clear. It is v..."
1999,"Mr. Speaker, according to \lineBreak The ques...","Mr. Speaker, in the past 18 months, Canada has..."


In [3]:

    
questions_file = 'context'
answers_file = 'answers'
vocabulary_file = 'vocabulary_movie'
padded_questions_file = 'Padded_context'
padded_answers_file = 'Padded_answers'
unknown_token = 'something'

# vocabulary_size = 7000
# max_features = vocabulary_size
# maxlen_input = 50
# maxlen_output = 50  # cut texts after this number of words

print ("Reading the context data...")
# q = open(questions_file, 'r')
questions = df_q_a['Q'].values
print ("Reading the answer data...")
# a = open(answers_file, 'r')
answers = df_q_a['A'].values

all_q_a = answers + questions
print ("Tokenazing the answers...")
paragraphs_a =  answers #[p for p in answers.split('\n')]
paragraphs_b =  questions # all_q_a #[p for p in all.split('\n')]
paragraphs_a = ['BOS '+p+' EOS' for p in paragraphs_a]
paragraphs_b = ['BOS '+p+' EOS' for p in paragraphs_b]

# update DF
df_q_a['paragraphs_a'] = paragraphs_a
df_q_a['paragraphs_b'] = paragraphs_b
df_q_a['Q_A'] = df_q_a[['paragraphs_a', 'paragraphs_b']].apply(lambda x: u' '.join(x), axis=1)


# paragraphs_b = ' '.join(paragraphs_b)
# tokenized_text = paragraphs_b.split()
# paragraphs_q = questions #[p for p in questions.split('\n') ]
tokenized_answers = [p.split() for p in paragraphs_a]
tokenized_questions = [p.split() for p in paragraphs_b]



### Counting the word frequencies:
##word_freq = nltk.FreqDist(itertools.chain(tokenized_text))
##print ("Found %d unique words tokens." % len(word_freq.items()))
##
### Getting the most common words and build index_to_word and word_to_index vectors:
##vocab = word_freq.most_common(vocabulary_size-1)
##
### Saving vocabulary:
##with open(vocabulary_file, 'w') as v:
##    pickle.dump(vocab, v)

print(answers[0])
print(tokenized_answers[0])
print(questions[0])
print(tokenized_questions[0])

df_q_a.tail()

Reading the context data...
Reading the answer data...
Tokenazing the answers...
Mr. Speaker, I want to thank the member for Huron—Bruce for his commitment to our great veterans and their families.  \lineBreak We are building on the successes of the privacy action plan to ensure that the private information of our veterans remains protected. That is why I present today the privacy action plan 2.0, which includes providing targeted training on privacy principles, streamlining consent forms and ensuring new initiatives are compliant with privacy requirements. \lineBreak Our government is clear: we will not tolerate any privacy breach. 
['BOS', 'Mr.', 'Speaker,', 'I', 'want', 'to', 'thank', 'the', 'member', 'for', 'Huron—Bruce', 'for', 'his', 'commitment', 'to', 'our', 'great', 'veterans', 'and', 'their', 'families.', '\\lineBreak', 'We', 'are', 'building', 'on', 'the', 'successes', 'of', 'the', 'privacy', 'action', 'plan', 'to', 'ensure', 'that', 'the', 'private', 'information', 'of', 'o

,Q,A,paragraphs_a,paragraphs_b,Q_A
1995,"Mr. Speaker, I can imagine the conversation th...","Mr. Speaker, we are the ones who put in place ...","BOS Mr. Speaker, we are the ones who put in pl...","BOS Mr. Speaker, I can imagine the conversatio...","BOS Mr. Speaker, we are the ones who put in pl..."
1996,"Mr. Speaker, Ms. Meilleur demonstrated that sh...","Mr. Speaker, after 10 years of petty politics ...","BOS Mr. Speaker, after 10 years of petty polit...","BOS Mr. Speaker, Ms. Meilleur demonstrated tha...","BOS Mr. Speaker, after 10 years of petty polit..."
1997,"Mr. Speaker, in Quebec, the law is clear. A ba...","Mr. Speaker, we want to ensure that Canadian c...","BOS Mr. Speaker, we want to ensure that Canadi...","BOS Mr. Speaker, in Quebec, the law is clear. ...","BOS Mr. Speaker, we want to ensure that Canadi..."
1998,"Mr. Speaker, I will now read a motion that the...","Mr. Speaker, we want to be very clear. It is v...","BOS Mr. Speaker, we want to be very clear. It ...","BOS Mr. Speaker, I will now read a motion that...","BOS Mr. Speaker, we want to be very clear. It ..."
1999,"Mr. Speaker, according to \lineBreak The ques...","Mr. Speaker, in the past 18 months, Canada has...","BOS Mr. Speaker, in the past 18 months, Canada...","BOS Mr. Speaker, according to \lineBreak The ...","BOS Mr. Speaker, in the past 18 months, Canada..."


In [4]:

max_words = 10000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100 

# print(u' '.join(str(v) for v in tokenized_questions))

print('Loading data...')
print('number of Q and A', len(df_q_a))

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_q_a[['Q_A']].apply(lambda x: u' '.join(x))) # fit on Q and A
X = tokenizer.texts_to_sequences(u' '.join(v) for v in tokenized_questions)
Y = tokenizer.texts_to_sequences(u' '.join(v) for v in tokenized_answers)


word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print ("Using vocabulary of size %d." % max_words)

X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
Y = pad_sequences(Y, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

print('X.shape', X.shape)
print('Y.shape', Y.shape)

print(X[0])

# Replacing all words not in our vocabulary with the unknown token:
# for i, sent in enumerate(tokenized_answers):
#     tokenized_answers[i] = [w if w in word_to_index else unknown_token for w in sent]

# for i, sent in enumerate(tokenized_questions):
#     tokenized_questions[i] = [w if w in word_to_index else unknown_token for w in sent]

# Creating the training data:
# X = np.asarray([[word_to_index[w] for w in sent] for sent in tokenized_questions])
# Y = np.asarray([[word_to_index[w] for w in sent] for sent in tokenized_answers])

# Q = sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
# A = sequence.pad_sequences(Y, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# print(Q[0])
# print(A[0])

with open(padded_questions_file, 'wb') as q:
    pickle.dump(X, q)
    
with open(padded_answers_file, 'wb') as a:
    pickle.dump(Y, a)
    
    
# Important.... print out the X and Y to find the EOS === here its 12
EOS = 12

Loading data...
number of Q and A 2000
Found 11495 unique tokens.
Using vocabulary of size 10000.
X.shape (2000, 100)
Y.shape (2000, 100)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0   11   14   13   23   21 1447  134 3427    4   23 2180  742    6
 1717  578  100   10  123  354    9  469  156    3  166    7  198   10
  296  397  742  156   92    2  121 2473 6936  268   15   79   45 3085
    1  297  189 2649  204 2650    3 1565    8  147    1   34  951    1
   62   22   86   23   21   28 1787    2 1054  365  337  742   15   42
  271   12]


In [5]:
# **********************************************************************
# Reading a pre-trained word embedding and addapting to our vocabulary:
# **********************************************************************

embeddings_index = {}
f = open( 'data/vectors.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('normalizing vectors')
embedding_matrix = preprocessing.robust_scale(embedding_matrix)

Found 429693 word vectors.
normalizing vectors


# Keras model of the chatbot: 

In [6]:

sentence_embedding_size = 300
weights_file = 'nothing yet'

def init_model():
    print(type(X), type(Y)) 

    input_context = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='input_context')
    input_answer = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='input_answer')

    LSTM_encoder = LSTM(sentence_embedding_size, kernel_initializer='lecun_uniform', name='encode_context')
    LSTM_decoder = LSTM(sentence_embedding_size, kernel_initializer='lecun_uniform', name='decode_answer')

    if os.path.isfile(weights_file):
        Shared_Embedding = Embedding(output_dim=EMBEDDING_DIM, input_dim=len(word_index) + 1, 
                                     input_length=MAX_SEQUENCE_LENGTH, name='shared_embedding')
    else:
    #     Shared_Embedding = Embedding(output_dim=EMBEDDING_DIM, input_dim=max_words, 
    #                                  weights=[embedding_matrix], input_length=maxlen_input)
        Shared_Embedding = Embedding(len(word_index) + 1, EMBEDDING_DIM,
                                    weights=[embedding_matrix],
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=False, name='shared_embedding')

    print('MAX_SEQUENCE_LENGTH', MAX_SEQUENCE_LENGTH)
    print('max_words', max_words)

    word_embedding_context = Shared_Embedding(input_context)
    context_embedding = LSTM_encoder(word_embedding_context)

    word_embedding_answer = Shared_Embedding(input_answer)
    answer_embedding = LSTM_decoder(word_embedding_answer)

    # merge_layer = Concatenate([context_embedding, answer_embedding], concat_axis=1)
    merge_layer = Concatenate(axis=1)([context_embedding, answer_embedding])

    out = Dense(int(max_words/2), activation='relu', name='relu_activation')(merge_layer)
    out = Dense(max_words, activation='softmax', name='softmax_activation')(out)

    model = Model(input=[input_context, input_answer], output = [out])

#     model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00005))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#     if os.path.isfile(weights_file):
#         model.load_weights(weights_file)

    model.summary()
    
    return model

model = init_model()

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
MAX_SEQUENCE_LENGTH 100
max_words 10000
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_context (InputLayer)      (None, 100)          0                                            
__________________________________________________________________________________________________
input_answer (InputLayer)       (None, 100)          0                                            
__________________________________________________________________________________________________
shared_embedding (Embedding)    (None, 100, 100)     1149600     input_context[0][0]              
                                                                 input_answer[0][0]               
__________________________________________________________________________________________________
encode_context (LSTM)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


In [7]:
# ************************************************************************
# Loading the data:
# ************************************************************************
n_test = 500
num_subsets = 5

q = X #cPickle.load(open(questions_file, 'rb'))
a = Y #cPickle.load(open(answers_file, 'rb'))

print(type(q))
print(a.shape)
# n_exem, n_words = a.shape
n_exem = a.shape[0]

qt = q[0:n_test,:]
at = a[0:n_test,:]
q = q[n_test + 1:,:]
a = a[n_test + 1:,:]

print(type(q))

print('Number of exemples = %d'%(n_exem - n_test))
print('qt', qt.shape, 'q', q.shape)
print('at', at.shape, 'a', a.shape)
step = int(np.around((n_exem - n_test)/num_subsets))
round_exem = int(step * num_subsets)
print('step', step, 'round_exem', round_exem)



<class 'numpy.ndarray'>
(2000, 100)
<class 'numpy.ndarray'>
Number of exemples = 1500
qt (500, 100) q (1499, 100)
at (500, 100) a (1499, 100)
step 300 round_exem 1500


In [8]:
Epochs = 100
BatchSize = 128  #  Check the capacity of your GPU
Patience = 0
dropout = .25
n_test = 100



In [18]:

vocabulary = dict((v, k) for k, v in tokenizer.word_index.items())
# vocabulary[1]

BOS = 11

def print_result(input_text):

    ans_partial = np.zeros((1,MAX_SEQUENCE_LENGTH))
    ans_partial[0, -1] = BOS  #  the index of the symbol BOS (begin of sentence)
    for k in range(MAX_SEQUENCE_LENGTH - 1):
        ye = model.predict([input_text, ans_partial])
        mp = np.argmax(ye)
        ans_partial[0, 0:-1] = ans_partial[0, 1:]
        ans_partial[0, -1] = mp
    
#     print(ans_partial)
    text = ''
    for k in ans_partial[0]:
        k = k.astype(int)
        if k < (max_words-2):
            w = vocabulary[k]
            text = text + w + ' '
    return(text)

def print_index_to_vocab(input_index):
    text = ''
    input_index = filter(lambda a: a != 0, input_index)
    for k in input_index:
        text = text + vocabulary[k] + ' '
    return(text)

    

# print(qt[0])
# print(at[0])
# print(q[0])
# print(a[0])

print(print_index_to_vocab(qt[41]))
test_input = at[41:42]
print(print_result(test_input))


# train_input = q[41:42]
# print(print_result(train_input)) 

bos order members know the standing orders require that we not interrupt other members when they are speaking linebreak the hon parliamentary secretary has the floor eos 
bos mr speaker the member for the member for the the member is the member for the linebreak we are the the member for the eos that is the the member for the the member for the eos that is the the member for the the member for the eos that is the the member for the the member for the eos that is the the member for the the member for the eos that is the the member for the the member for the eos that is the the member for the the member for the eos that is 


In [10]:
# *************************************************************************
# Bot training:
# *************************************************************************
print(round_exem, step)

x = range(0,Epochs) 
valid_loss = np.zeros(Epochs)
train_loss = np.zeros(Epochs)
for m in range(Epochs):
    
    # Loop over training batches due to memory constraints:
    for n in range(0,round_exem,step):
        print('n', n)
        
        q2 = q[n:n+step]
        s = q2.shape
        count = 0
        for i, sent in enumerate(a[n:n+step]):
            l = np.where(sent==EOS)  #  the position od the symbol EOS
            limit = l[0][0]
            count += limit + 1
            
        print(count)
        Q = np.zeros((count,MAX_SEQUENCE_LENGTH))
        A = np.zeros((count,MAX_SEQUENCE_LENGTH))
        Y = np.zeros((count,max_words))
        
        # Loop over the training examples:
        count = 0
        for i, sent in enumerate(a[n:n+step]):
            ans_partial = np.zeros((1,MAX_SEQUENCE_LENGTH))
            
            # Loop over the positions of the current target output (the current output sequence):
            l = np.where(sent==EOS)  #  the position of the symbol EOS
            limit = l[0][0]

            for k in range(1,limit+1):
                # Mapping the target output (the next output word) for one-hot codding:
                y = np.zeros((1, max_words))
                y[0, sent[k]] = 1

                # preparing the partial answer to input:

                ans_partial[0,-k:] = sent[0:k]

                # training the model for one epoch using teacher forcing:
                
                Q[count, :] = q2[i:i+1] 
                A[count, :] = ans_partial 
                Y[count, :] = y
                count += 1
                
        print('Training epoch: %d, training examples: %d - %d'%(m,n, n + step))
        model.fit([Q, A], Y, batch_size=BatchSize, epochs=1, verbose=1)
         
        test_input = qt[41:42]
        print(print_result(test_input))
        train_input = q[41:42]
        print(print_result(train_input))        
        
    model.save_weights(weights_file, overwrite=True)


1500 300
n 0
22009
Training epoch: 0, training examples: 0 - 300
Epoch 1/1
22009/22009 [==============================] - 302s 14ms/step - loss: 6.4447
bos mr speaker the member the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member 
bos mr speaker the member the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of the member of 

bos mr speaker i am proud of the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for the member for 
n 900
21397
Training epoch: 1, training examples: 900 - 1200
Epoch 1/1
21397/21397 [==============================] - 297s 14ms/step - loss: 4.8684
bos mr speaker i am very proud to the member for the member for linebreak the member for linebreak the member for linebreak the member for linebreak the member for the member for linebreak the member for linebreak the member for linebreak the member for the member for linebreak the member for linebreak the member for linebreak the member for the member fo

n 0
22009
Training epoch: 3, training examples: 0 - 300
Epoch 1/1
22009/22009 [==============================] - 299s 14ms/step - loss: 4.7174
bos mr speaker the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the 
bos mr speaker the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the member opposite is the member opposite that the me

bos mr speaker i am pleased to thank the member for the member opposite that the ndp has been the member opposite that the ndp has been the ndp members are the member to be the member for the eos of the ndp has been a very clear that the ndp has been the ndp to be the ndp and the ndp members are the of the ndp and the ndp members are the of the ndp and the ndp members are the of the ndp and the ndp members are the of the ndp and the ndp members are 
n 600
21208
Training epoch: 4, training examples: 600 - 900
Epoch 1/1
21208/21208 [==============================] - 273s 13ms/step - loss: 4.3676
bos mr speaker i am pleased to the member that the ndp has been a very clear that we are going to do eos and we have been working with the provinces and territories and we are working with the provinces and territories and we are going to do eos and we have been working with the provinces and territories and we are going to do eos and we have been working with the provinces and territories and we

bos mr speaker i am pleased to thank the member for linebreak the member opposite has been to the member opposite has been to the member opposite is to the same time we are going to make sure that we have the same time to ensure that we are going to make sure that we have the same time to ensure that we are going to make sure that we have the same time to ensure that we are going to make sure that we have the same time to ensure that we are going to make sure that we 
n 1200
21949
Training epoch: 5, training examples: 1200 - 1500
Epoch 1/1
21949/21949 [==============================] - 283s 13ms/step - loss: 4.1569
bos mr speaker the member opposite is the member for linebreak the member opposite is the member for linebreak the member opposite is the member for linebreak the member opposite is the member for linebreak the member opposite is the member for linebreak the member opposite is the member for linebreak the member opposite is the member for linebreak the member opposite is the

Training epoch: 7, training examples: 300 - 600
Epoch 1/1
21352/21352 [==============================] - 290s 14ms/step - loss: 3.9126
bos mr speaker i am pleased to thank the member for linebreak the ndp members know that the ndp has been the ndp members are not the eos to the ndp members have been very clear that the ndp has been the ndp members are not the eos to the ndp members have been very clear that the ndp has been the ndp members are not the eos to the ndp members have been very clear that the ndp has been the ndp members are not the eos to the ndp members have been very clear that the ndp has 
bos mr speaker i am pleased to thank the member for linebreak i would like to thank the member for linebreak i would like to thank the member for linebreak i would like to thank the member for linebreak i would like to thank the member for linebreak i would like to thank the member for linebreak i would like to thank the member for linebreak i would like to thank the member for linebre

bos mr speaker i am pleased to thank my colleague for his own question linebreak i am very pleased to thank the member for linebreak i am very pleased to thank the member for linebreak i am very pleased to thank the member for linebreak i am very pleased to thank the member for linebreak i am very pleased to thank the member for linebreak i am very pleased to thank the member for linebreak i am very pleased to thank the member for linebreak i am very pleased to thank the member for linebreak i am very pleased to 
n 1200
21949
Training epoch: 8, training examples: 1200 - 1500
Epoch 1/1
21949/21949 [==============================] - 296s 13ms/step - loss: 3.6137
bos mr speaker i am very pleased to be very clear that the ndp did not understand what there were not just as the ndp members are not working with indigenous peoples linebreak we have done a strong way to ensure that we have the best way to ensure that all of the canadian armed forces have the best way to ensure that they are not

KeyboardInterrupt: 